In [1]:
# Download data fro3m http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# %lprun -f

In [2]:
import numpy as np
import re
import collections
import operator
from scipy.optimize import minimize
import copy

from scipy.optimize import fmin_bfgs

In [3]:
def findDic(text):
    words = re.compile('[a-zA-Z\']*[a-zA-Z]').findall(text)
    return collections.Counter(words)

In [4]:
def readData(directory, paths, limit=None):
    if limit:
        paths = paths[0:limit]

    data = []
    for path in paths:
        txt = open(directory + path).read()
        data.append(txt)
    return data
        
        
negDataPath = !ls data/neg/
negData = readData('./data/neg/', negDataPath, 100)

posDataPath = !ls data/pos/
posData = readData('./data/pos/', posDataPath, 100)

In [5]:
def findWordCount(data):
    wordCount = []
    for review in data:
        wordCount.append(findDic(review))
    return wordCount

negDataDic = findWordCount(negData)
posDataDic = findWordCount(posData)

In [6]:
def updateDic(mainDic, dicArray):
    mainDic = copy.deepcopy(mainDic)
    for dic in dicArray:
        mainDic.update(dic)
    
    # update indexes
    for index, key in enumerate(mainDic):
        mainDic[ key ] = index
        
    return mainDic

allWords = {}
allWords = updateDic(allWords, negDataDic)
allWords = updateDic(allWords, posDataDic)

In [7]:
def findRow(words, text, boolean=False):
    row = np.zeros(len(allWords))
    wordsDic = findDic(text)
    for word in words:
        if word in allWords:
            row[allWords[word]] = 1 if boolean else wordsDic[word]
    return row

In [8]:
def createTable(allWords, negData, y0, posData, y1):
    x = []
    for data in negData + posData:
        x.append( findRow(allWords, data) )
    x = np.row_stack(x)
    
    y = np.append(y0 * np.ones(len(negData)), y1 * np.ones(len(posData)))
    return x, y

In [9]:
x, y = createTable(allWords, negData, -1, posData, 1)

In [ ]:
# def cost(w, b, c, x, y):
#     p = 1 - y * (x.dot(w) + b)
#     p[p<0] = 0
#     p = p**2
#     return w.dot(w.T) + c * p.sum()

In [ ]:
# M = len(allWords)
# w = np.zeros(M)
# b = 0
# c = 100

# cost(w, b, c, x, y)

In [ ]:
# M = len(allWords)
# w = np.zeros(M)
# b = 0
# c = 100
 
# Nfeval = 1
# print '{0:s}  {1:s}'.format('iter', 'cost')
# def callbackF(Xi):
#     global Nfeval
#     global b, c, x, y
#     print '{0:4d}  {1:f}'.format(Nfeval, cost(np.array(Xi), b, c, x, y))
#     Nfeval += 1
    
# res = minimize(cost, w, method='Nelder-Mead', args=(b, c, x, y), options={'xtol': 1e-8, 'disp': True, 'maxiter': 5}, callback=callbackF)
# res.x

In [16]:
# def newCost(w):
#     return .5 * w.dot(w.T)

In [15]:
# def newConstraint(w, y, x, b, index):
#     return int( y[index] * (x[index].dot(w) + b) - 1 )

In [ ]:
# w = np.zeros( len(allWords) )
# b = 0
# cons = [{'type': 'ineq', 'fun': newConstraint, 'args': (ny, nx, b, index)} for index in range(len(x))]

In [ ]:
# Nfeval = 1
# print '{0:s}  {1:s}'.format('iter', 'cost')
# def callbackF(Xi):
#     global Nfeval
#     print '{0:4d}  {1:f}'.format(Nfeval, newCost(np.array(Xi)))
#     Nfeval += 1
    
# solution = minimize(newCost, w, method='SLSQP', constraints=cons, callback=callbackF)